<a href="https://colab.research.google.com/github/MD-Daniel99/Projects/blob/main/team_project_ver_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install sentencepiece sacremoses transformers langid
!pip install --upgrade python-docx
!pip install --upgrade pdfminer.six

import langid
from transformers import pipeline
from transformers import AutoTokenizer

from docx import Document
from pdfminer.high_level import extract_text

## File type recognition

In [2]:
filename = "/content/Black_body_radiation(test.docx"

In [3]:
def filetype_recognition(filename):
  if filename.lower().endswith('.docx'):
    return docx_reader(filename)
  elif filename.lower().endswith('.pdf'):
    return pdf_reader(filename)
  elif filename.lower().endswith('.txt'):
    return txt_reader(filename)
  else:
    raise ValueError("Unsupported file format. Please, make sure you download .docx, .pdf or .txt.")

## .docx reader

In [4]:
def docx_reader(filename):
  doc = Document(filename)
  text = "\n".join([paragraph.text for paragraph in doc.paragraphs])
  str(text)
  return text

## .pdf reader


In [5]:
def pdf_reader(filename):
  text = extract_text(filename)
  str(text)
  return text

## .txt reader

In [6]:
def txt_reader(filename):
  with open(filename, 'r') as file_object:
    text = file_object.read()
    str(text)
  return text

## Summary model start-up

In [15]:
def trim_text_to_token_limit(content, max_token_limit = 512, model_name="Falconsai/text_summarization"):
    # Загрузка токенизатора для заданной модели
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Токенизация текста
    tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(content)))

    # Обрезка текста до заданного количества токенов
    trimmed_tokens = tokens[:max_token_limit]

    # Обратная токенизация для получения текста
    trimmed_text = tokenizer.decode(tokenizer.convert_tokens_to_ids(trimmed_tokens))

    return trimmed_text

In [8]:
def summary_model(trimmed_text):
  contents = pipeline('summarization', model = "Falconsai/text_summarization")
  summarised_text = contents(trimmed_text, min_length = 70)[0]
  return summarised_text['summary_text']

## En-Ru translation model start-up

In [9]:
def model_loading():
  classifier = pipeline("translation_en_to_ru", model = "Helsinki-NLP/opus-mt-en-ru")
  return classifier

In [17]:
def translation(classifier, summary):
  sentences = summary.split('. ')
  translated_sentences = [classifier(sentence)[0]['translation_text'] for sentence in sentences]
  return '. '.join(translated_sentences)

In [11]:
def printing(translated):
  output_file = '/content/processed_text.txt'
  with open(output_file, 'w') as file:
    file.write(translated)

In [12]:
def file_preprocessing(filename):
    file_content = filetype_recognition(filename)
    return file_content.rstrip()

In [ ]:
def main():
  try:
    content = file_preprocessing(filename)
    classifier = model_loading()
    trimmed_text = trim_text_to_token_limit(content)
    summary = summary_model(trimmed_text)
    #prompt = input("Choose whether you want to translate the text:")
    prediction = translation(classifier, summary)
    printing(prediction)

  except ValueError as ve:
    print(ve)
    return

if __name__ == "__main__":
    main()

In [ ]:
main()